# Hopfield Network With Hashing - Hopfield

This is a memory mechanism in a form of a Hopfield network. The stored items are called memory patterns. They are retrieved by a process of the input that is presented to the network dynamics which at some time step reaches a fixed stable point. This means that the input item has been recognized (i.e. there is a memory pattern identical or very similar to it).

Even noisy sounds or those corrupted to some extent can be accessed. In other words, if the input is $x_1 + \delta$ and the stored item is $x_1$, the network will still reach the fixed point of $x_1$ if $\delta$ is small enough.

Additionally, for storage purposes, sounds are transformed each into a hash - with this we reduce their dimensionality. This means we increase the storage capacity. 

### Load dependencies

In [1]:
# First, we load the neccessary dependencies.

import numpy as np
import math
import librosa
import scipy.io.wavfile as wav
import sys
import glob
import pickle
import random
import matplotlib.pyplot as plt

In [2]:
# Folder with some wav files to test this script.

folder_train = "./wavs/"

test_folder = "./test_wavs/"

### Extract Features

First, we will transform our .wav files into features, in this case MFCCs.

In [11]:
def make_mfcc(folder):
    """
    Go through the folder and find all (and only) files ending with .wav
    Here, we transform each .wav file into MFCCs and then flatten them into one vector.
    We do this because we want one hash per .wav file.
    
    Any file shorter than the longest file in the folder will be padded with values 0,
    so that all concatenated file vectors are of the same length.
    
    Parameters
    ----------
    folder : path to folder with wav sounds
    
    Returns
    -------
    a list of flattened MFCC vectors
    """
    vectors = []
    for file in glob.glob(folder + "*.wav", recursive=True):
        y, sr = librosa.load(file)
        mfcc_feat = librosa.feature.mfcc(y=y, sr=sr)
        vect = mfcc_feat.flatten()
        vectors.append(vect)
        print(len(vectors), " mfccs done")
    # find the largest vector
    max_length = len(max(vectors, key=lambda p: len(p)))
    # append zeros to all the other vectors
    for i in range(len(vectors)):
        vectors[i] = np.pad(vectors[i], (0,max_length-len(vectors[i])))
    return vectors

### Hashing of features

Now we will use these features and transform them into hash vectors, which we will use to store in our memory. We do this to facilitate memory storage: hashes are vectors with reduced dimensionality, with values mostly equal to 0 and a few of them equal to 1.

In [4]:
def hash_dim(d,k,m,seed):  
    
    """
    Define hash parameters.
    The hash will be a matrix of the dimension = k*m
    We choose a random number k of units of the vector.
    
    Parameters
    ----------
    d : num
        Length of a random vector being stored
    k : num
        Number of units we randomly choose of the vector
    m : num
        Number of times we will  do the hashing for some vector
    seed : num
        We always want the same units randomly chosen
        
    Returns
    -------
    a numpy array 
        p of dimensions [k,m] represents randomly chosen dimensions
    
    """   
    assert k <= d
    p = np.zeros((m,k,))
    np.random.seed(seed)
    for i in range(m):
        p[i] = np.random.permutation(d)[:k]
    return p

    
def get_hash(vector, k, m, p): 
    """
    Transform a vector of speech into a hash
    The hash will be a matrix of the dimension = k*m
    
    Once we have chosen k random dimensions, we look for the highest 
    value and turn it into 1. Everything else is 0.
    We thus get sparse matrices.
    We do this m times. Final output is h=k*m.
    
    Parameters
    ----------
    vector : np.array
        Features (i.e. MFCC) of some sound with dim = 1*n
    k : num
        Number of units we randomly choose of the vector
    m : num
        Number of times we will do the hashing for some vector.
    p : numpy array
        p of dimensions [k,m] represents randomly chosen dimensions
        
    Returns
    -------
    a numpy array h of size [1, k*m]
    """
    h = np.zeros((m,k,))
    for i in range(m):
        p_line = p[i]
        ix = np.argmax(vector[p_line])
        hi = np.zeros(k)
        hi[ix] = 1
        h[i] = hi
    h = np.hstack(h)
    return h

In [5]:
# TEST

expected_h = np.array([[1,0,0],[0,0,1]]).flatten()
vector = np.array([6,4,5,9,2])
# %timeit hash_dim(len(vector),3,2,2).astype(int)
p0 = hash_dim(len(vector),3,2,2).astype(int)
print("This is a test hash: ", get_hash(vector, 3, 2, p0))
assert get_hash(vector, 3, 2, p0).all() == expected_h.all()

This is a test hash:  [1. 0. 0. 0. 0. 1.]


### Memory storage

We must now construct our neuron weight matrix that reopresents the connections between neurons of our memory network.
We will first initialize our matrix representing the synaptic weights and then enable subsequent addition of new memories.

Synaptic weight matrix is a matrix that represents connections between each and every neuron. Every neuron has a state which can be active or inactive. Initialization of synaptic weights will make the connection between two neurons such that it is strengthened if both neurons are active (and the other way round, it will weaken the connection if one of the neurons is inactive).

In [6]:
def initialize_network(N, V=None):
    """
    Eq. (2) from [1]
    
    Initialize synaptic weights in form of matrix T (symmetric recurrent weight matrix).
    This is a memory storage.
    
    Parameters
    ----------
    N : num
        number of neurons
    V : list
        list of vectors in a hash form
        
    Returns
    -------
    a numpy array T of shape (N, N)
        Memory storage in form of a matrix (synaptic weights)
        Its dimensions are determined by N=k*m (hash parameters)
    """
    
    %timeit
    T = np.zeros((N,N))
    if V != None:
        for vect in V:
            #outer_prod = np.outer((2*vect - 1),(2*vect - 1))
            T = add_memory(T, vect)
            #T += outer_prod
        
    return T


def add_memory(T, new_memory):
    """
    Eq. (2) from [1]
    
    Update synaptic weights in form of matrix T (symmetric recurrent weight matrix) when adding new memory.
    
    Parameters
    ----------
    T : a numpy array T_sum of shape (N, N)
        Initialized memory storage in form of a matrix (synaptic weights)
    new_memory : numpy array of shape (1,N)
        a vector we wish to store
        
    Returns
    -------
    a numpy array T of shape (N, N)
        Renewed memory storage in form of a matrix (synaptic weights)
        Its dimensions are determined by N=k*m (hash parameters)
    """
    
    outer_prod = np.outer((2*new_memory - 1),(2*new_memory - 1))
    T += outer_prod
        
    return T

### Memory retrieval

When we want to retrieve a memory, we start with some initial state and then observe the dynamics of the system - if it reaches a stable point, we have accessed to either some stored memory or to a state by default where we can end up if we have not stored something similar to initial state.

In other words, we can represent this as a surface with differently sized bumps. We put a ball on this surface and it will roll into the nearest pit, unless we already put it on the already lowest point of the pit.

To check whether this lowest point (or stable/fixed point) was reached, we check stability of being there - have we been here a few moments ago? 

In [7]:
def energy(T, V):
    """
    Eq. (7) from [1]
    
    Energy of the system is a monotonically decreasing function.
    
    Parameters
    ----------
    T : a numpy array of shape (N, N)
        Memory in form of a matrix (synaptic weights)
    V : numpy array
        a list of states of activation
        
    Returns
    -------
    num
        Energy of the system
    """
    E = 0
    for i in range(T.shape[0]):
        for j in range(T.shape[1]):
            E -= 1/2 * (T[i,j] * V[i] * V[j])
    return E

In [8]:
def retrieve_memory(T, V0, U=0, full_trace=True, SEED=27, conv_check_spacing=100):
    """
    Eq. (1) from [1]
    
    To retrieve a memory, we want to find the stable/fixed point of the 
    dynamic network represented by matrix T (synaptic weights in which
    the memory is stored) when starting from vector V.

    Parameters
    ----------
    T : a numpy array T of shape (N, N)
        Memory in form of a matrix (synaptic weights)
    V0 : a numpy array of shape (1, N)
        a vector with which we initialize the network activity (check if it is stored in T)
    U : num
        a scalar representing the threshold of neuron's state of activity.
        Set to 0 by default.
        "With a threshold of 0,the system behaves as a forced categorizer." [1]
    full_trace : boolean
        Set to True by default. This means we will keep all the changes of the initial neuron states
        as they change through time.
    SEED : num
        Used for the random choices of indices, which we can control for replication by always 
        setting the seed to the same number.
        
    Returns
    -------
    a numpy array of shape (1, N)
        We return the new / denoised V
    """
    random.seed(SEED)
    
    V = V0
    if full_trace:
        V_history = [V.copy()]
    
    while not has_converged(T, U, V):
        for _ in range(conv_check_spacing):
            i = random.randrange(V.shape[0])
            V[i] = update_neuron(T, U, V, i)
            if full_trace:
                V_history.append(V.copy())

    if full_trace:
        return V_history
    else:
        return V


def update_neuron(T, U, V, i):
    """
    We calculate sum_j {T_ji * V[j]}, where T_ij is our synaptic weight between neuron j and i
    and V[j] is a j-th neuron state 
    
    Parameters
    ----------
    T : a numpy array T_sum of shape (N, N)
        Initialized memory storage in form of a matrix (synaptic weights)
    V : num
        a scalar representing the neural network state
        
    Returns
    -------
    num
        We return the sum of all components of i-th row of T, each multiplied by V_j
    """
    new_V_i = check_threshold(sum(T[i] * V), U)
    return new_V_i



def check_threshold(membrane_potential, U):
    """
    Check whether the sum of T_ij * V_j is bigger or smaller than the threshold U

    Parameters
    ----------
    TV_sum : num
        Sum over T_ij * V_j
    U : num
        a scalar representing a threshold of neuron's state of activity
        
    Returns
    -------
    num
        We return either 0 or 1, depending on TV_sum being smaller
        or larger than the threshold U
    """
    if membrane_potential > U:
        return 1
    else:
        return 0
    
    
def has_converged(T, U, V):
    """
    Check whether the new V_i is the same as i-th value of V
    and whether the current energy is equal to the previous one

    Parameters
    ----------
    V_i : num
        state of V's i-th neuron (either active: 1, or inactive: 0)
    V : num
        a numpy array representing all current neurons' states of activity
    i : num
        current randomly chosen index
    E_list : list
        list of all energy values so far
        
    Returns
    -------
    boolean
        We return False if we satisfy at least one of the conditions,
        else we return True
    """
    converged = True
    for i, V_i in enumerate(V):
        updated_V_i = update_neuron(T, U, V, i)
        if updated_V_i != V_i:
            converged = False
            break
    return converged    

In [10]:
# TEST

toy_V1 = np.array([0,1])
toy_V2 = np.array([1,0])
# %timeit initialize_network(2, [toy_V1, toy_V2])
toy_T = initialize_network(2, [toy_V1, toy_V2])
print("Toy T matrix: \n", toy_T)
print("Retrieved memory of [0,0] - unstored:\n", retrieve_memory(toy_T, np.array([0,0])))
print("Retrieved memory of [1,1] - unstored:\n", retrieve_memory(toy_T, np.array([1,1])))
print("Retrieved memory of [9,9] - unstored:\n", retrieve_memory(toy_T, np.array([9,9])))
print("Retrieved memory of [0,1] - stored:\n", retrieve_memory(toy_T, toy_V1))

Toy T matrix: 
 [[ 2. -2.]
 [-2.  2.]]
Retrieved memory of [0,0] - unstored:
 [array([0, 0])]
Retrieved memory of [1,1] - unstored:
 [array([1, 1]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0]), array([1, 0

### Testing 

We can now inspect our memory network and test it:

In [27]:
# REAL VECTORS, ALL FILES

mfccs_vectors = make_mfcc(folder_train)
# store the mfccs as a pickle file 
with open("vectors.txt", "wb") as fp:
     pickle.dump(mfccs_vectors, fp)

1  mfccs done
2  mfccs done
3  mfccs done
4  mfccs done
5  mfccs done
6  mfccs done
7  mfccs done
8  mfccs done
9  mfccs done
10  mfccs done
11  mfccs done
12  mfccs done
13  mfccs done
14  mfccs done
15  mfccs done
16  mfccs done
17  mfccs done
18  mfccs done
19  mfccs done
20  mfccs done
21  mfccs done
22  mfccs done
23  mfccs done
24  mfccs done
25  mfccs done
26  mfccs done
27  mfccs done
28  mfccs done
29  mfccs done
30  mfccs done
31  mfccs done
32  mfccs done
33  mfccs done
34  mfccs done
35  mfccs done
36  mfccs done
37  mfccs done
38  mfccs done
39  mfccs done
40  mfccs done
41  mfccs done
42  mfccs done
43  mfccs done
44  mfccs done
45  mfccs done
46  mfccs done
47  mfccs done
48  mfccs done
49  mfccs done
50  mfccs done
51  mfccs done
52  mfccs done
53  mfccs done
54  mfccs done
55  mfccs done
56  mfccs done
57  mfccs done
58  mfccs done
59  mfccs done
60  mfccs done
61  mfccs done
62  mfccs done
63  mfccs done
64  mfccs done
65  mfccs done
66  mfccs done
67  mfccs done
68  

520  mfccs done
521  mfccs done
522  mfccs done
523  mfccs done
524  mfccs done
525  mfccs done
526  mfccs done
527  mfccs done
528  mfccs done
529  mfccs done
530  mfccs done
531  mfccs done
532  mfccs done
533  mfccs done
534  mfccs done
535  mfccs done
536  mfccs done
537  mfccs done
538  mfccs done
539  mfccs done
540  mfccs done
541  mfccs done
542  mfccs done
543  mfccs done
544  mfccs done
545  mfccs done
546  mfccs done
547  mfccs done
548  mfccs done
549  mfccs done
550  mfccs done
551  mfccs done
552  mfccs done
553  mfccs done
554  mfccs done
555  mfccs done
556  mfccs done
557  mfccs done
558  mfccs done
559  mfccs done
560  mfccs done
561  mfccs done
562  mfccs done
563  mfccs done
564  mfccs done
565  mfccs done
566  mfccs done
567  mfccs done
568  mfccs done
569  mfccs done
570  mfccs done
571  mfccs done
572  mfccs done
573  mfccs done
574  mfccs done
575  mfccs done
576  mfccs done
577  mfccs done
578  mfccs done
579  mfccs done
580  mfccs done
581  mfccs done
582  mfc

1032  mfccs done
1033  mfccs done
1034  mfccs done
1035  mfccs done
1036  mfccs done
1037  mfccs done
1038  mfccs done
1039  mfccs done
1040  mfccs done
1041  mfccs done
1042  mfccs done
1043  mfccs done
1044  mfccs done
1045  mfccs done
1046  mfccs done
1047  mfccs done
1048  mfccs done
1049  mfccs done
1050  mfccs done
1051  mfccs done
1052  mfccs done
1053  mfccs done
1054  mfccs done
1055  mfccs done
1056  mfccs done
1057  mfccs done
1058  mfccs done
1059  mfccs done
1060  mfccs done
1061  mfccs done
1062  mfccs done
1063  mfccs done
1064  mfccs done
1065  mfccs done
1066  mfccs done
1067  mfccs done
1068  mfccs done
1069  mfccs done
1070  mfccs done
1071  mfccs done
1072  mfccs done
1073  mfccs done
1074  mfccs done
1075  mfccs done
1076  mfccs done
1077  mfccs done
1078  mfccs done
1079  mfccs done
1080  mfccs done
1081  mfccs done
1082  mfccs done
1083  mfccs done
1084  mfccs done
1085  mfccs done
1086  mfccs done
1087  mfccs done
1088  mfccs done
1089  mfccs done
1090  mfccs do

In [ ]:
# TEST VECTORS, 4 FILES

mfccs_vectors = make_mfcc(test_folder)
# store the mfccs as a pickle file 
with open("test_vectors.txt", "wb") as fp:
     pickle.dump(mfccs_vectors, fp)

In [30]:
with open("vectors.txt", "rb") as fp:
     mfccs_vectors = pickle.load(fp)
      
    
# V_train = V[:len(V)//2]
# V_test = V[len(V)//2:]
# T = initialize_network(N, V_train)

In [33]:
# Test with random dimensions k, m, N, which determine the size of T and of hash vectors:

def test_parameters(mfccs_vectors, k, m):
    N = k*m
    V =[]
    p = hash_dim(N,k,m,27).astype(int)
    for vect in mfccs_vectors:
        v = get_hash(vect, k, m, p)
        V.append(v) 
    return V

In [38]:
# def precision_recall_(T, V, N, U):
#     correct = 0
#     incorrect = 0
#     default = np.zeros(N)
#     for v in V:
#         memory = retrieve_memory(T, v, U=U, full_trace=False)
#         if np.array_equal(memory, default):
#             correct += 1
#         else:
#             incorrect += 1
#     correct = correct/len(V)
#     incorrect = incorrect/len(V)
#     return [correct, incorrect]

# H = precision_recall_(T, V_test, N, U)[0]
# FA = precision_recall_(T, V_test, N, U)[1]
# CR = precision_recall_(T, V_train, N, U)[0]
# M = precision_recall_(T, V_train, N, U)[1]
# print(H, FA, CR, M)

def precision_recall(T, V_train, V_test, N, U):
    
    hits = 0
    false_alarm = 0
    default = np.zeros(N)
    for v in V_test:
        memory = retrieve_memory(T, v, U=U, full_trace=False)
        if np.array_equal(memory, default):
            hits += 1
        else:
            false_alarm += 1
    hits = hits/len(V_test)
    false_alarm = false_alarm/len(V_test)

    correct_rejection = 0
    miss = 0
    default = np.zeros(N)
    for v in V_train:
        memory = retrieve_memory(T, v, U=U, full_trace=False)
        if np.array_equal(memory, default):
            miss += 1
        else:
            correct_rejection += 1
    correct_rejection = correct_rejection/len(V_train)
    miss = miss/len(V_train)
    
    result = [hits, false_alarm, correct_rejection, miss]
    return result

In [43]:
for k in range(2,11):
    for m in range(2,11):
        U = 0
        print(k, m)
        N = k*m
        V = test_parameters(mfccs_vectors, k, m)
        V_train = V[:len(V)//2]
        V_test = V[len(V)//2:]
        T = initialize_network(N, V_train)
        print(precision_recall(T, V_train, V_test, N, U))

2 2
[0.0, 1.0, 1.0, 0.0]
2 3
[0.0, 1.0, 1.0, 0.0]
2 4
[0.0, 1.0, 1.0, 0.0]
2 5
[0.0, 1.0, 1.0, 0.0]
2 6
[0.0, 1.0, 1.0, 0.0]
2 7
[0.0, 1.0, 1.0, 0.0]
2 8
[0.0, 1.0, 1.0, 0.0]
2 9
[0.0, 1.0, 1.0, 0.0]
2 10
[0.0, 1.0, 1.0, 0.0]
3 2
[0.0, 1.0, 1.0, 0.0]
3 3
[0.0, 1.0, 1.0, 0.0]
3 4
[0.0, 1.0, 1.0, 0.0]
3 5
[0.0, 1.0, 1.0, 0.0]
3 6
[0.0, 1.0, 1.0, 0.0]
3 7
[0.0, 1.0, 1.0, 0.0]
3 8
[0.0, 1.0, 1.0, 0.0]
3 9
[0.0, 1.0, 1.0, 0.0]
3 10
[0.0, 1.0, 1.0, 0.0]
4 2
[0.0, 1.0, 1.0, 0.0]
4 3
[0.0, 1.0, 1.0, 0.0]
4 4
[0.0, 1.0, 1.0, 0.0]
4 5
[0.0, 1.0, 1.0, 0.0]
4 6
[0.0, 1.0, 1.0, 0.0]
4 7
[0.0, 1.0, 1.0, 0.0]
4 8
[0.0, 1.0, 1.0, 0.0]
4 9
[0.0, 1.0, 1.0, 0.0]
4 10
[0.0, 1.0, 1.0, 0.0]
5 2
[0.0, 1.0, 1.0, 0.0]
5 3
[0.0, 1.0, 1.0, 0.0]
5 4
[0.0, 1.0, 1.0, 0.0]
5 5
[0.0, 1.0, 1.0, 0.0]
5 6
[0.0, 1.0, 1.0, 0.0]
5 7
[0.0, 1.0, 1.0, 0.0]
5 8
[0.0, 1.0, 1.0, 0.0]
5 9
[0.0, 1.0, 1.0, 0.0]
5 10
[0.0, 1.0, 1.0, 0.0]
6 2
[0.0, 1.0, 1.0, 0.0]
6 3
[0.0, 1.0, 1.0, 0.0]
6 4
[0.0, 1.0, 1.0, 0.0]
6 5
[0.0, 1.0, 1.0, 0

In [41]:
# We can inspect how one of our hashed vectors looks like:

print(V[0])

[0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 1. 1. 0. 0. 0.]


We can also see here how our matrix T (symmetric recurrent weight matrix) looks like:

In [42]:
# We can inspect how our transition matrix looks like:

print(T)

[[ 600. -434. -272. ...  150. -204. -176.]
 [-434.  600.  254. ...  -20.  338.  318.]
 [-272.  254.  600. ...  150.  488.  224.]
 ...
 [ 150.  -20.  150. ...  600.  198.  -78.]
 [-204.  338.  488. ...  198.  600.  276.]
 [-176.  318.  224. ...  -78.  276.  600.]]


Now we will test our memory with the stored and unstored data. We want to see whether the system finds the stored memories and whether it finds the general fixed point (only 0s) for the unstored ones.

In [46]:
V_unstored_1 = np.array([3, 1, 4, 1, 0, 0, 0, 0, 7, 0, 9, 0, 7, 0, 4])
V_unstored_2 = np.array([1,1,1,1,1,1,0,0,0,0,0,0,0,0,0])

print(retrieve_memory(T, V[1])[-1])
print(retrieve_memory(T, V_unstored_1)[-1])
print(retrieve_memory(T, V_unstored_2)[-1])

[0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.]
[1 1 1 0 1 1 1 1 0 1 1 1 1 1 0]
[1 1 1 0 1 1 1 1 0 1 1 1 1 1 0]


In [48]:
from sklearn.metrics import average_precision_score
average_precision_unstored = average_precision_score(retrieve_memory(T, V_unstored_1)[-1], V_unstored_1)
average_precision_stored = average_precision_score(retrieve_memory(T, V[1])[-1], V[1])

print('Precision-recall score for the unstored: {0:0.2f}'.format(
      average_precision_unstored))
print('Precision-recall score for the stored: {0:0.2f}'.format(
      average_precision_stored))

Precision-recall score for the unstored: 1.00
Precision-recall score for the stored: 1.00


### References

[1] Hopfield, J. J. (1982). Neural networks and physical systems with emergent collective computational abilities. Proceedings of the national academy of sciences, 79(8), 2554-2558. 
[https://doi.org/10.1073/pnas.79.8.2554]

[2] Andoni, A., & Indyk, P. (2006, October). Near-optimal hashing algorithms for approximate nearest neighbor in high dimensions. In 2006 47th annual IEEE symposium on foundations of computer science (FOCS'06) (pp. 459-468). IEEE.
[https://10.1109/FOCS.2006.49]

In [ ]:
# TEST
# robustness
# precision/recall
# size of input that can be stocked
# nature of stimuli
# nature of hash
# put silence at the end so all have the same length
# link to ref - this equation num. from this paper